In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np

from category_encoders.target_encoder import TargetEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

In [2]:
train = pd.read_csv('application_train.csv')
test = pd.read_csv('application_test.csv')

In [3]:
import analytics as als

In [4]:
df = train.append(test).reset_index()

In [5]:
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])

In [6]:
import analytics as als

In [7]:
feature_str, feature_int = als.get_lists_type_feature(df, list(df))

In [8]:
df = pd.get_dummies(df, columns = feature_str, dummy_na = True)

In [9]:
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)

In [10]:
df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

In [11]:
train = df[df['TARGET'].notnull()]
test = df[df['TARGET'].isnull()]

In [19]:
# df.to_csv('train2.csv', sep=';', index=False)

In [13]:
train.columns = ["".join (i if i.isalnum() else "_" for i in str(x)) for x in train.columns]

In [14]:
X = train.drop(columns=['SK_ID_CURR', 'TARGET'])
y = train['TARGET']

In [15]:
kf = StratifiedKFold(n_splits=5,  shuffle=True)

In [16]:
feature_str, feature_int = als.get_lists_type_feature(X, list(X))

In [17]:
for feature in feature_int:
    X[feature] = X[feature].astype('float64')

In [20]:
for train_indices, test_indices in kf.split(X, y):
    X_train, X_val = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_val = y.iloc[train_indices], y.iloc[test_indices]
    lgbmc = lgb.LGBMClassifier(nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1)
    model = lgbmc.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    score_train = roc_auc_score(y_train, y_pred_train)
    print(f'train_score {score_train}')   
    y_pred_val = model.predict(X_val)
    score_val = roc_auc_score(y_val, y_pred_val)
    print(f'test_score {score_val}')

train_score 0.6421831719929383
test_score 0.5195659011360324


KeyboardInterrupt: 